In [ ]:
from pypdf import PdfReader
reader = PdfReader('Sanskrit_Text (1).pdf')
page = reader.pages[1]
print(page.extract_text((0, 90)))


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# Load image
img = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg') 

# Create Gabor filter bank
gabor_kernels = cv2.getGaborKernel((5, 5), 4.0, theta=0, lambd=10.0, gamma=0.5)

# Filter image with Gabor filters
gabor_imgs = []
for kernel in gabor_kernels:
    filtered_img = cv2.filter2D(img, -1, kernel)
    gabor_imgs.append(filtered_img)

# Detect edges
edges = cv2.Canny(gabor_imgs[0], 50, 200) 

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength=10, maxLineGap=5)

# Draw bounding boxes on original image
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.rectangle(img, (x1+15, y1), (x2+15, y2), (0, 255, 0), 2)

cv2.imwrite('output.jpg', img)

Working

In [23]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load image
img = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg') 

# Create Gabor filter bank
gabor_kernels = cv2.getGaborKernel((5, 5), 4.0, theta=0, lambd=10.0, gamma=0.5)

# Filter image with Gabor filters
gabor_imgs = []
for kernel in gabor_kernels:
    filtered_img = cv2.filter2D(img, -1, kernel)
    gabor_imgs.append(filtered_img)

# Detect edges
edges = cv2.Canny(gabor_imgs[0], 50, 200)

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength=10, maxLineGap=5)

# Group lines based on y-coordinates
lines_sorted = sorted(lines, key=lambda line: line[0][1])  # Sort lines based on y-coordinates
grouped_lines = []
current_group = [lines_sorted[0]]
for i in range(1, len(lines_sorted)):
    if abs(lines_sorted[i][0][1] - lines_sorted[i-1][0][1]) < 10:  # Tweak the threshold value for grouping
        current_group.append(lines_sorted[i])
    else:
        grouped_lines.append(current_group)
        current_group = [lines_sorted[i]]
grouped_lines.append(current_group)

# Draw bounding boxes on original image for each text line
for line_group in grouped_lines:
    min_x = min(line[0][0] for line in line_group)
    min_y = min(line[0][1] for line in line_group)
    max_x = max(line[0][2] for line in line_group)
    max_y = max(line[0][3] for line in line_group)
    cv2.rectangle(img, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)

cv2.imwrite('11.jpg', img)


True

Tweak the below

In [3]:
import json
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load image
img = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg') 

# Create Gabor filter bank
# Apply Gabor filter
ksize = 31  # Size of the Gabor filter kernel (adjust based on image size and line thickness)
sigma = 200  # Standard deviation of the Gaussian kernel (adjust based on image size and line thickness)
theta = np.pi/4  # Orientation of the Gabor filter (horizontal)
lambda_ = 0.1  # Wavelength of the sinusoidal factor (adjust based on line thickness)
gamma = 0.1  # Spatial aspect ratio (adjust to get the desired response)
gabor_kernels = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambda_, gamma)


filtered_img = cv2.filter2D(img, cv2.CV_8UC3, gabor_kernels)

# Filter image with Gabor filters
gabor_imgs = []
for kernel in gabor_kernels:
    filtered_img = cv2.filter2D(img, -1, kernel)
    gabor_imgs.append(filtered_img)

# Detect edges
edges = cv2.Canny(gabor_imgs[0], 50, 200)

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength=7, maxLineGap=8)

# Group lines based on y-coordinates
lines_sorted = sorted(lines, key=lambda line: line[0][1])  # Sort lines based on y-coordinates
grouped_lines = []
current_group = [lines_sorted[0]]
for i in range(1, len(lines_sorted)):
    if abs(lines_sorted[i][0][1] - lines_sorted[i-1][0][1]) < 4.5:  # Tweak the threshold value for grouping
        current_group.append(lines_sorted[i])
    else:
        grouped_lines.append(current_group)
        current_group = [lines_sorted[i]]
grouped_lines.append(current_group)
# Create a dictionary to store bounding box coordinates
bbox_dict = {}

# Draw bounding boxes on original image for each text line and save coordinates to the dictionary
for idx, line_group in enumerate(grouped_lines, start=1):
    min_x = min(line[0][0] for line in line_group)
    min_y = min(line[0][1] for line in line_group)
    max_x = max(line[0][2] for line in line_group)
    max_y = max(line[0][3] for line in line_group)

    # Convert numpy.int32 to regular Python int
    min_x, min_y, max_x, max_y = int(min_x), int(min_y), int(max_x), int(max_y)

    # Store coordinates in the dictionary
    box_name = f"box{idx}"
    bbox_dict[box_name] = {
        "top_left": [min_x, min_y],
        "top_right": [max_x, min_y],
        "bottom_left": [min_x, max_y],
        "bottom_right": [max_x, max_y]
    }

    # Draw the bounding box on the image
    cv2.rectangle(img, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)

    # Optionally, you can print the coordinates for each line
    print(f"Line {idx}: ({min_x}, {min_y}), ({max_x}, {max_y})")

# Save the dictionary as a JSON file
json_file_path = 'bounding_boxes.json'
with open(json_file_path, 'w') as json_file:
    json.dump(bbox_dict, json_file, indent=4)

cv2.imwrite('11.jpg', img)

Line 1: (255, 245), (341, 280)
Line 2: (310, 272), (322, 273)
Line 3: (333, 278), (333, 252)
Line 4: (311, 290), (1097, 361)
Line 5: (255, 371), (498, 410)
Line 6: (311, 424), (1142, 499)
Line 7: (255, 519), (341, 544)
Line 8: (255, 546), (341, 553)
Line 9: (311, 568), (1104, 642)
Line 10: (272, 1179), (650, 1283)
Line 11: (307, 1291), (525, 1266)
Line 12: (276, 1309), (532, 1345)
Line 13: (633, 1368), (641, 1391)


True


# Saved Images using the following --- Last tried

In [5]:
import os
import json
import cv2
# Load image
img_path = '/home/kalyan/gitrepo/NeedToStartARepo/iitb/123.png'
img = cv2.imread(img_path) 

# Create Gabor filter bank
gabor_kernels = cv2.getGaborKernel((5, 5), 4.0, theta=0, lambd=10.0, gamma=0.5)

# Filter image with Gabor filters
gabor_imgs = []
for kernel in gabor_kernels:
    filtered_img = cv2.filter2D(img, -1, kernel)
    gabor_imgs.append(filtered_img)

# Detect edges
edges = cv2.Canny(gabor_imgs[0], 50, 200)

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength=10, maxLineGap=5)

# Group lines based on y-coordinates
lines_sorted = sorted(lines, key=lambda line: line[0][1])  # Sort lines based on y-coordinates
grouped_lines = []
current_group = [lines_sorted[0]]
for i in range(1, len(lines_sorted)):
    if abs(lines_sorted[i][0][1] - lines_sorted[i-1][0][1]) < 6:  # Tweak the threshold value for grouping
        current_group.append(lines_sorted[i])
    else:
        grouped_lines.append(current_group)
        current_group = [lines_sorted[i]]
grouped_lines.append(current_group)

# Create a new folder with the same name as the original image
img_folder = os.path.splitext(os.path.basename(img_path))[0]  # Get the image file name without extension
output_folder = os.path.join(os.path.dirname(img_path), img_folder)
os.makedirs(output_folder, exist_ok=True)

# Create a dictionary to store bounding box coordinates
bbox_dict = {}

def convert_np_int_to_int(obj):
    if isinstance(obj, np.int32):
        return int(obj)
    elif isinstance(obj, list):
        return [convert_np_int_to_int(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_np_int_to_int(value) for key, value in obj.items()}
    else:
        return obj
# Draw bounding boxes on original image for each text line and save coordinates to the dictionary
for idx, line_group in enumerate(grouped_lines, start=1):
    min_x = min(line[0][0] for line in line_group)
    min_y = min(line[0][1] for line in line_group)
    max_x = max(line[0][2] for line in line_group)
    max_y = max(line[0][3] for line in line_group)
    cv2.rectangle(img, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)

    # Save the cropped and boxed image in the new folder
    cropped_img = img[min_y:max_y, min_x:max_x]
    if not cropped_img.any():  # Check if cropped_img is empty
        print(f"Warning: Cropped image is empty for box {idx}. Skipping...")
    else:
        boxed_img_path = os.path.join(output_folder, f"boxed_line_{idx}.jpg")
        cv2.imwrite(boxed_img_path, cropped_img)

        # Store coordinates in the dictionary
        box_name = f"box{idx}"
        bbox_dict[box_name] = {
            "top_left": [min_x, min_y],
            "top_right": [max_x, min_y],
            "bottom_left": [min_x, max_y],
            "bottom_right": [max_x, max_y]
        }

# Save the dictionary as a JSON file in the same folder as the original image
json_file_path = os.path.join(output_folder, 'bounding_boxes.json')
with open(json_file_path, 'w') as json_file:
    json.dump(convert_np_int_to_int(bbox_dict), json_file, indent=4)


# Save the modified image with bounding boxes
cv2.imwrite(os.path.join(output_folder, 'Overall.jpg'), img)


True

Play with Gabor and put parameters above

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

def gabor_filter(size, theta, frequency, sigma_x, sigma_y):
    """Generate a Gabor filter."""
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)
    g = np.exp(-0.5 * (x_theta * 2 / sigma_x * 2 + y_theta * 2 / sigma_y * 2)) * np.cos(2 * np.pi * frequency * x_theta)
    return g / np.max(g)  # Normalize the filter to have values between 0 and 1

# Parameters for Gabor filter
filter_size = 31
theta = np.pi / 4
frequency = 200
sigma_x = 0.1
sigma_y = 0.1

# Create a Gabor filter
gabor = gabor_filter(filter_size, theta, frequency, sigma_x, sigma_y)

# Load your bw image here or replace 'image' with your actual image data
image = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg') 
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Apply the Gabor filter to the image using convolution
filtered_image = convolve2d(image, gabor, mode='same', boundary='symm')

# Plot the original image
#plt.figure(figsize=(10, 10))
#plt.imshow(image, cmap='gray', alpha=1.0)
#
## Plot the Gabor filter on top of the image with transparency
#plt.imshow(gabor, cmap='gray', alpha=0.5, extent=[0, image.shape[1], image.shape[0], 0], interpolation='bilinear')
#
#plt.axis('off')
#plt.show()

def move_gabor_filter(gabor_filter, x, y):
    """Move the Gabor filter to the specified (x, y) position."""
    filter_size = gabor_filter.shape[0]
    x_min, x_max = max(0, x), min(gabor_filter.shape[1], x + filter_size)
    y_min, y_max = max(0, y), min(gabor_filter.shape[0], y + filter_size)
    moved_filter = np.zeros_like(gabor_filter)
    src_x_min, src_x_max = x_min - x, x_max - x
    src_y_min, src_y_max = y_min - y, y_max - y
    moved_filter[y_min:y_max, x_min:x_max] = gabor_filter[src_y_min:src_y_max, src_x_min:src_x_max]
    return moved_filter

# Example usage:
# (Assuming 'gabor' is your Gabor filter generated earlier and 'x' and 'y' are the translation coordinates)

x = 10  # Move the filter 10 pixels to the right
y = 5  # Move the filter 20 pixels down

moved_gabor = move_gabor_filter(gabor, x, y)

# Plot moved Gabor filter
plt.figure(figsize=(10, 10))
plt.imshow(moved_gabor, cmap='gray', alpha=1.0)
plt.axis('off')
plt.show()


Tried to increase height , without changing width -- Old one

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load image
img = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg') 

# Create Gabor filter bank
gabor_kernels = cv2.getGaborKernel((5, 5), 4.0, theta=0, lambd=10.0, gamma=0.5)

# Filter image with Gabor filters
gabor_imgs = []
for kernel in gabor_kernels:
    filtered_img = cv2.filter2D(img, -1, kernel)
    gabor_imgs.append(filtered_img)

# Detect edges
edges = cv2.Canny(gabor_imgs[0], 50, 200) 

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength=10, maxLineGap=5)

# Draw bounding boxes on original image with increased height
for line in lines:
    x1, y1, x2, y2 = line[0]
    
    # Increase the height of the bounding box by adding an offset to y-coordinates
    y_offset = 10  # Adjust this value to control the increase in height
    y1_new = max(0, y1 - y_offset)
    y2_new = min(img.shape[0] - 1, y2 + y_offset)
    
    cv2.rectangle(img, (x1, y1_new), (x2, y2_new), (0, 255, 0), 2)

cv2.imwrite('output_with_increased_height.jpg', img)



Play with Gabor and see it's plot

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load image
img_path = '/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg'
img = cv2.imread(img_path)

# Convert image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gabor filter
ksize = 31  # Size of the Gabor filter kernel (adjust based on image size and line thickness)
sigma = 5   # Standard deviation of the Gaussian kernel (adjust based on image size and line thickness)
theta = 90  # Orientation of the Gabor filter (horizontal)
lambda_ = 10  # Wavelength of the sinusoidal factor (adjust based on line thickness)
gamma = 0.5  # Spatial aspect ratio (adjust to get the desired response)
gabor_filter = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambda_, gamma)
filtered_img = cv2.filter2D(gray_img, cv2.CV_8UC3, gabor_filter)

# Find contours of horizontal lines in the filtered image
contours, _ = cv2.findContours(filtered_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get the bounding box of the highest horizontal line
max_y = 0
bbox = None
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if h > w:  # Consider only horizontal lines (adjust this condition if needed)
        if y > max_y:
            max_y = y
            bbox = (x, y, w, h)

#draw bounding box
x, y, w, h = bbox
cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the original image and the image with bounding box
plt.subplot(121), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
plt.plot(122), plt.imshow(cv2.cvtColor(filtered_img, cv2.COLOR_BGR2RGB)), plt.title('Image with Bounding Box')
plt.show()

SVD

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load image
img = cv2.imread('/home/kalyan/gitrepo/NeedToStartARepo/iitb/ilovepdf_pages-to-jpg/Sanskrit_Text_page-0001.jpg')

# Convert image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Perform Singular Value Decomposition (SVD)
U, S, Vt = np.linalg.svd(gray_img)

# Choose the number of singular values to use for reconstruction
num_singular_values = 250

# Reconstruct the image using selected singular values
reconstructed_img = np.dot(U[:, :num_singular_values], np.dot(np.diag(S[:num_singular_values]), Vt[:num_singular_values, :]))

# Convert the reconstructed image back to uint8 (8-bit integer)
reconstructed_img = np.uint8(reconstructed_img)

# Use OpenCV to find contours in the image
contours, _ = cv2.findContours(reconstructed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw bounding boxes around detected objects
img_with_bboxes = img.copy()
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(img_with_bboxes, (x, y), (x + w, y + h), (0, 255, 0), 2)

# plot the original image and the reconstructed image
plt.subplot(121), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original Image')
plt.subplot(122), plt.imshow(cv2.cvtColor(reconstructed_img, cv2.COLOR_BGR2RGB)), plt.title('Reconstructed Image')
plt.show()

#save the reconstructed image
cv2.imwrite('reconstructed_img.jpg', reconstructed_img)